# Support Vector Machine (SVM) vs Decision  Tree
### This notebook uses svm and decision tree algorithims to find the best classification model for this data set. 

<br/>Data Source: 
https://archive.ics.uci.edu/ml/datasets/SGEMM+GPU+kernel+performance

In [7]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score  
from sklearn.tree import DecisionTreeClassifier

seed = 20200222

file1 = r'sgemm_product.csv'

# https://archive.ics.uci.edu/ml/datasets/SGEMM+GPU+kernel+performance

## Read Data in and Prep it

In [8]:
# Read and prepare sgemm_product data 
df_one_01 = pd.read_csv(file1)
print(df_one_01.shape)

#Add an average of the Runs
df_one_01['RunAvg'] = (df_one_01['Run1 (ms)']+df_one_01['Run2 (ms)']+df_one_01['Run3 (ms)']+df_one_01['Run4 (ms)'])/4
print(df_one_01.shape,'Added Average Column')

df_one_01['B_RunAvg'] = np.where(df_one_01['RunAvg'] < df_one_01['RunAvg'].mean(), 0, 1)
print(df_one_01.shape,'Turned RunAvg into a Binary Class')

#Drop columns not needed
df_one_01.drop(labels=['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)','RunAvg']
            , axis=1
            , inplace=True)
print(df_one_01.shape,'Data has been read in.')
print(df_one_01.head())

(241600, 18)
(241600, 19) Added Average Column
(241600, 20) Turned RunAvg into a Binary Class
(241600, 15) Data has been read in.
   MWG  NWG  KWG  MDIMC  NDIMC  MDIMA  NDIMB  KWI  VWM  VWN  STRM  STRN  SA  \
0   16   16   16      8      8      8      8    2    1    1     0     0   0   
1   16   16   16      8      8      8      8    2    1    1     0     0   0   
2   16   16   16      8      8      8      8    2    1    1     0     0   1   
3   16   16   16      8      8      8      8    2    1    1     0     0   1   
4   16   16   16      8      8      8      8    2    1    1     0     1   0   

   SB  B_RunAvg  
0   0         0  
1   1         0  
2   0         0  
3   1         0  
4   0         0  


In [9]:
# Get Smaller data set for SVM
df_one_01 = df_one_01.sample(n=10000, random_state=1)
print(df_one_01.shape)
print(df_one_01.head())

(10000, 15)


        MWG  NWG  KWG  MDIMC  NDIMC  MDIMA  NDIMB  KWI  VWM  VWN  STRM  STRN  \
106120   64   64   32     16     16      8     32    8    4    1     1     0   
128737   64  128   32      8     16     32      8    2    1    4     0     0   
123689   64  128   32      8      8      8     16    2    2    4     1     0   
208174  128  128   16      8     32     16     16    2    2    2     1     1   
239379  128  128   32     32      8     32     16    2    2    2     0     0   

        SA  SB  B_RunAvg  
106120   0   0         0  
128737   0   1         0  
123689   0   1         1  
208174   1   0         1  
239379   1   1         0  


## Split Train and Test Data sets

In [10]:
# ==============================================================================
# Create a Training and Testing data set
# ==============================================================================
X = df_one_01.drop(['B_RunAvg'], axis=1)
y = df_one_01['B_RunAvg']

d1_X_train, d1_X_test, d1_y_train, d1_y_test = train_test_split(X, y, test_size=0.30, random_state=seed)

print('d1_X_train\t',d1_X_train.shape,'\t', len(d1_X_train)/len(df_one_01))
print('d1_y_train\t',d1_y_train.shape,'\t', len(d1_y_train)/len(df_one_01))
print('d1_X_test\t',d1_X_test.shape,'\t', len(d1_X_test)/len(df_one_01))
print('d1_y_test\t',d1_y_test.shape,'\t', len(d1_y_test)/len(df_one_01),'\n')


d1_X_train	 (7000, 14) 	 0.7
d1_y_train	 (7000,) 	 0.7
d1_X_test	 (3000, 14) 	 0.3
d1_y_test	 (3000,) 	 0.3 



## Run a cross validation to score each model

In [11]:
%%time 
#  10K took 33 Sec
D1_SVML = []
D1_SVMR = []
D1_SVMP = []
D1_DT1 = []
D1_DT2 = []
D1_DT3 = []
D1_DT4 = []
D1_DT5 = []


D1_SVML.append(cross_val_score(estimator=SVC(kernel='linear'),X=d1_X_train,y=d1_y_train,cv=5))
D1_SVMR.append(cross_val_score(estimator=SVC(kernel='rbf'),X=d1_X_train,y=d1_y_train,cv=5))
D1_SVMP.append(cross_val_score(estimator=SVC(kernel='poly'),X=d1_X_train,y=d1_y_train,cv=5))
D1_DT1.append(cross_val_score(estimator=DecisionTreeClassifier(criterion='gini'),X=d1_X_train,y=d1_y_train,cv=5))
D1_DT2.append(cross_val_score(estimator=DecisionTreeClassifier(criterion='gini',max_depth=4),X=d1_X_train,y=d1_y_train,cv=5))
D1_DT3.append(cross_val_score(estimator=DecisionTreeClassifier(criterion='gini',max_depth=6),X=d1_X_train,y=d1_y_train,cv=5))
D1_DT4.append(cross_val_score(estimator=DecisionTreeClassifier(criterion='gini',max_depth=8),X=d1_X_train,y=d1_y_train,cv=5))
D1_DT5.append(cross_val_score(estimator=DecisionTreeClassifier(criterion='gini',max_depth=10),X=d1_X_train,y=d1_y_train,cv=5))

print("D1 SVM kernel='linear':\t", np.mean(D1_SVML))
print("D1 SVM kernel='rbf':\t", np.mean(D1_SVMR))
print("D1 SVM kernel='poly':\t", np.mean(D1_SVMP))
print("D1 DT1:\t", np.mean(D1_DT1))
print("D1 DT2:\t", np.mean(D1_DT2))
print("D1 DT3:\t", np.mean(D1_DT3))
print("D1 DT4:\t", np.mean(D1_DT4))
print("D1 DT5:\t", np.mean(D1_DT5))


D1 SVM kernel='linear':	 0.9301428571428572
D1 SVM kernel='rbf':	 0.9001428571428571
D1 SVM kernel='poly':	 0.9029999999999999
D1 DT1:	 0.9585714285714285
D1 DT2:	 0.8677142857142858
D1 DT3:	 0.9078571428571429
D1 DT4:	 0.939857142857143
D1 DT5:	 0.9577142857142856
Wall time: 53.3 s


## Best Model Test

In [12]:
%%time 
# Fit the SVM model to eash 
ker = 'linear'

# SVM ---------------------------------------------------------------------
svm_Model = SVC(kernel=ker)
svm_Model.fit(d1_X_train, d1_y_train)
svm_Test_pred_y = svm_Model.predict(d1_X_test)

SVM_Confusion_Matrix = pd.DataFrame(data=confusion_matrix(d1_y_test,svm_Test_pred_y)
                                    # ,index =['Class 1 True','Class 2 True','Class 3 True','Class 4 True']
                                    # ,columns =['Pred 1', 'Pred 2', 'Pred 3', 'Pred 4'])
                                    , index =['Actual No', 'Actual Yes']
                                    , columns =['Prediced No','Predicted Yes'])
SVM_CR=classification_report(d1_y_test,svm_Test_pred_y)
MSE_SVM = (np.square(d1_y_test - svm_Test_pred_y)).mean()
SSE_SVM = (np.square(d1_y_test - svm_Test_pred_y)).sum()

print('SVM MSE',MSE_SVM)
print('SVM SSE',SSE_SVM)
print(SVM_Confusion_Matrix,'\n')
print(SVM_CR,'\n\n')

# DT ---------------------------------------------------------------------
dt_Model = DecisionTreeClassifier()
dt_Model.fit(d1_X_train, d1_y_train)
dt_Test_pred_y = dt_Model.predict(d1_X_test)

dt_Confusion_Matrix = pd.DataFrame(data=confusion_matrix(d1_y_test,dt_Test_pred_y)
                                    # ,index =['Class 1 True','Class 2 True','Class 3 True','Class 4 True']
                                    # ,columns =['Pred 1', 'Pred 2', 'Pred 3', 'Pred 4'])
                                    , index =['Actual No', 'Actual Yes']
                                    , columns =['Prediced No','Predicted Yes'])
dt_CR=classification_report(d1_y_test,dt_Test_pred_y)
MSE_dt = (np.square(d1_y_test - dt_Test_pred_y)).mean()
SSE_dt = (np.square(d1_y_test - dt_Test_pred_y)).sum()

print('DT MSE',MSE_dt)
print('DT SSE',SSE_dt)
print(dt_Confusion_Matrix,'\n')
print(dt_CR,'\n\n')

SVM MSE 0.068
SVM SSE 204
            Prediced No  Predicted Yes
Actual No          2149             88
Actual Yes          116            647 

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      2237
           1       0.88      0.85      0.86       763

    accuracy                           0.93      3000
   macro avg       0.91      0.90      0.91      3000
weighted avg       0.93      0.93      0.93      3000
 


DT MSE 0.04066666666666666
DT SSE 122
            Prediced No  Predicted Yes
Actual No          2171             66
Actual Yes           56            707 

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2237
           1       0.91      0.93      0.92       763

    accuracy                           0.96      3000
   macro avg       0.94      0.95      0.95      3000
weighted avg       0.96      0.96      0.96      3000
 


Wall time: 13.1 s


### Both models preforemed well. The best model was the descion tree. 
SVM accuray = .93
</br>Decision Tree accuray = .96